In [ ]:
import os
import numpy as np 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import plot, init_notebook_mode, iplot
import plotly.express as px

init_notebook_mode(connected=True)
from vae_dist.dataset.fields import mat_pull, split_and_filter

import numpy as np 
from vae_dist.dataset.fields import mat_pull
from vae_dist.data.augmentation import Augment

In [ ]:
def plot_vfield(mat, cutoff = 95, min_max = True, scale = 10):
        x = mat #dataset_test.data[0] 
        x = x.reshape(50, 50, 50, 3)
        
        u_1, v_1, w_1 = split_and_filter(
            x, cutoff=cutoff, std_mean=False, min_max=False
        )
        a, b, c = np.meshgrid(
                np.arange(-5.0, 5.0, 0.2),
                np.arange(-5.0, 5.0, 0.2),
                np.arange(-5.0, 5.0, 0.2)
                )

        #max value of each dimension
        max_u = np.max(u_1)
        max_v = np.max(v_1)
        max_w = np.max(w_1)
        print(max_u, max_v, max_w)
        
        cones = go.Cone(
                x=a.flatten(), 
                y=b.flatten(), 
                z=c.flatten(), 
                u=u_1 ,
                v=v_1 , 
                w=w_1 ,
                sizeref=scale,)
                #opacity=0.0) 
                
        layout = go.Layout(
                title='Cones',
                width=700,
                height=700,
                        #sizeref=0.5,
                        #anchor='tail'
                )

        fig = go.Figure(data=cones,layout=layout)
        iplot(fig)


def write_mat(mat, filename, start_line='', spacing = 0.3):
    # write seven empty lines 

    with open(filename, 'w') as f:
        for i in range(6):
            if (start_line != '' and i == 0):
                f.write(start_line)
            f.write("\n")
        
        for i in range(mat.shape[0]):
            for j in range(mat.shape[1]):
                for k in range(mat.shape[2]):
                    # round to 2 decimal places
                    i_ = np.around(i - np.floor(mat.shape[0]/2), decimals=4)
                    j_ = np.around(j - np.floor(mat.shape[1]/2), decimals=4)
                    k_ = np.around(k - np.floor(mat.shape[2]/2), decimals=4)
                    #print(mat[i][j][k])
                    f.write("{:.3f} {:.3f} {:.3f} {:.6f} {:.6f} {:.6f}\n".format(
                        i_*spacing, j_*spacing, k_*spacing, 
                        mat[i][j][k][0], mat[i][j][k][1], mat[i][j][k][2])
                    )

In [ ]:
meta= {'steps_x': 50, 'steps_y': 50, 'steps_z': 50, 'step_size_x': 0.2, 'step_size_y': 0.2, 'step_size_z': 0.2, 'first_line': '#Sample Density: 25 25 25; Volume: Box: 5.000000 5.000000 5.000000\n'}
mat_aug_reshape = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))

up = np.array([0, 0, 1.0])
down = np.array([0, 0, -1.0])
left = np.array([0, -1.0, 0])
right = np.array([0, 1.0, 0])
front = np.array([-1.0, 0, 0])
back = np.array([1.0, 0, 0])

mat_up = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_down = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_left = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_right = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_front = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_back = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_sink = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))
mat_source = np.zeros((meta["steps_x"], meta['steps_y'], meta['steps_z'], 3))

mat_up[:, :, :, :] = up
mat_down[:, :, :, :] = down
mat_left[:, :, :, :] = left
mat_right[:, :, :, :] = right
mat_front[:, :, :, :] = front
mat_back[:, :, :, :] = back

# get center index of the matrix
center = np.array([meta["steps_x"], meta['steps_y'], meta['steps_z']]) / 2
print(center)
# get direction to origin at each point


benchmarks = [mat_up, mat_down, mat_left, mat_right, mat_front, mat_back]
file_list = ['up', 'down', 'left', 'right', 'front', 'back']

In [ ]:
output_dir = "./baseline_fields/"

for i, mat in enumerate(benchmarks):
    file = file_list[i]
    write_mat(
        mat,
        output_dir + file.split(".")[0] + ".dat",
        start_line = meta['first_line'],
        spacing = meta['step_size_x'])

In [ ]:
original_field = mat_pull("./baseline_fields/left.dat")
original_field = original_field.reshape(1, 3, 50, 50, 50)

In [ ]:
plot_vfield(
        original_field,  
        cutoff = 0,
        scale = 0.7)

In [ ]:
field_0 = mat_pull("./augmented_baselines/left_0.dat")
field_1 = mat_pull("./augmented_baselines/left_1.dat")
field_2 = mat_pull("./augmented_baselines/left_2.dat")
field_3 = mat_pull("./augmented_baselines/left_3.dat")
field_4 = mat_pull("./augmented_baselines/left_4.dat")
field_5 = mat_pull("./augmented_baselines/left_5.dat")
field_6 = mat_pull("./augmented_baselines/left_6.dat")

In [ ]:
plot_vfield(
        field_0,  
        cutoff = 0,
        scale = 0.7)

In [ ]:
plot_vfield(
        field_1,  
        cutoff = 0,
        scale = 0.7)

In [ ]:
plot_vfield(
        field_3,  
        cutoff = 0,
        scale = 0.7)

In [4]:
import torch 
from vae_dist.dataset.dataset import FieldDatasetSupervised
root = "../data/cpet/"
supervised_file = "../data/protein_data.csv"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_vanilla = FieldDatasetSupervised(
    root, 
    supervised_file,
    transform=False, 
    augmentation=False,
    standardize=True,
    lower_filter=True,
    log_scale=False, 
    min_max_scale=True,
    wrangle_outliers=True,
    scalar=False,
    device=device, 
    offset=1
)


wrangle data mean 1.1772095293703486, std 6.992790413113798
wrangle min 0.0008873698092109061, max 4370.6400039582795
22.15558076871174
Data shape:  (186, 3, 21, 21, 21)
Data type:  float64
------------------------- Preprocessing Info -------------------------
Helmholtz-Hodge decomposition applied:  False
Lower filter applied:  True
Log scale applied:  False
Standardization applied:  True
Min max scaling applied:  True
Wrangling outliers applied:  True
------------------------- Data Info -------------------------
Mean value in dataset:  -0.008502826629883897
Standard deviation in dataset:  0.05034438558765044
Largest value in dataset:  0.88
Smallest value in dataset:  -0.97
Nan values in dataset:  False
Inf values in dataset:  False


In [9]:
dataset_vanilla[[0,2,3]][1].shape

torch.Size([3, 3])

In [19]:
from torch.nn import functional as F
import torch
input = torch.randn(3, 5, requires_grad=True)
print(input.shape)
print(input.dtype)
target = torch.randint(5, (3,), dtype=torch.int64)
print(target.shape)
print(target.dtype)
loss = F.cross_entropy(input, target)

torch.Size([3, 5])
torch.float32
torch.Size([3])
torch.int64
